In [ ]:
!pip install fastapi uvicorn
!pip install pydantic

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import requests
from PIL import Image
import io
import numpy as np

In [ ]:
app = FastAPI()

In [ ]:
class InputData(BaseModel):
    image_url: str
    number: int
    text: str

In [ ]:
def process_data(image: Image.Image, number: int, text: str):
    img_array = np.array(image.resize((28, 28)).convert('L')) / 255.0

    score = np.mean(img_array) + number * 0.1 + len(text) * 0.05

    label = "Aprobado" if score > 1 else "Rechazado"

    return {"score": round(score, 2), "label": label}


In [ ]:
@app.post("/predict")
async def predict(data: InputData):
    response = requests.get(data.image_url)

    image = Image.open(io.BytesIO(response.content))

    result = process_data(image, data.number, data.text)

    return result
